In [24]:
import pandas as pd
import re
import requests
from lxml import html
import sys

In [25]:
class Scrapper:

    def __init__(self, source, link_pattern, xpath, df):
        self.source = source
        self.link_pattern = link_pattern
        self.xpath = xpath
        self.df = df
        self.link_storage = []
        self.link_history = []
        
    def extract_links(self, page):
        pattern = self.source + '[a-z0-9\-\_\.\/]+'
        links = re.findall(pattern, page.text)
        for link in links:
            if link not in self.link_storage:
                self.link_storage.append(link)
        return links

    def load(self, link):
        response = requests.get(link)
        return response

    def is_needed(self, link):
        return bool(re.search(self.link_pattern, link))
        
    def extract_info(self, page):
        tree = html.fromstring(page.content)
        result = tree.xpath(self.xpath)
        self.df = self.df.append(pd.DataFrame(result, columns=list(self.df.columns)), ignore_index=True)
        print('Dobavil')
        return result


    def scrape(self, seed_link):
        self.link_storage.append(seed_link)
        while True:
            links_list_length = len(self.link_storage)
            links = self.link_storage.copy()
            for link in links:
                if link not in self.link_history:
                    seed_page = self.load(link)
                    if seed_page:
                        print('Loaded ' + link)
                        if self.is_needed(link):
                            self.extract_info(seed_page)
                        self.extract_links(seed_page)
                    if self.df.shape[0] > 20:
                        break
                if links_list_lenth == len(self.link_storage):
                    
                    
            if self.df.shape[1] > 2:
                break
        return self.df
            


In [17]:
sys.setrecursionlimit(100)

df = pd.DataFrame(columns=['title'])

scrapper_kloop = Scrapper('https://www.bbc.com/news/world', 
                          'com/news/', 
                        '//h1/text()', 
                          df)


In [18]:
scrapper_kloop.extract_links(scrapper_kloop.load(scrapper_kloop.source))

['https://www.bbc.com/',
 'https://www.bbc.com/news',
 'https://www.bbc.com/sport',
 'https://www.bbc.com/weather',
 'https://www.bbc.com/reel/',
 'https://www.bbc.com/travel/',
 'https://www.bbc.com/capital/',
 'https://www.bbc.com/culture/',
 'https://www.bbc.com/future/',
 'https://www.bbc.com/food/',
 'https://www.bbc.com/bitesize',
 'https://www.bbc.com/nature/',
 'https://www.bbc.com/wwscripts/data',
 'https://www.bbc.com/wwscripts/flag',
 'https://www.bbc.com/food',
 'https://www.bbc.com/bitesize',
 'https://www.bbc.com/earth',
 'https://www.bbc.com/news/uk-45952532',
 'https://www.bbc.com/news/uk-45952532',
 'https://www.bbc.com/news/world-us-canada-46070329',
 'https://www.bbc.com/news/world-us-canada-46070329',
 'https://www.bbc.com/reel',
 'https://www.bbc.com/reel/video/p06p7fp7/the-slave-ship-hidden-for-200-years',
 'https://www.bbc.com/reel/',
 'https://www.bbc.com/reel/video/p06p7fp7/the-slave-ship-hidden-for-200-years',
 'https://www.bbc.com/reel/video/p06fshzv/five-com

In [19]:
#scrapper_kloop.link_storage

In [7]:
#page = scrapper_kloop.load('https://kloop.kg/blog/2018/11/01/isa-omurkulov-bolshe-ne-chlen-sdpk-no-ostaetsya-liderom-ee-parlamentskoj-fraktsii-chto-proishodit/')

In [8]:
#scrapper_kloop.extract_info(page)

In [9]:
#scrapper_kloop.is_needed('https://www.bbc.com/news/world-middle-east-46067959')

In [10]:
#df.shape()

In [11]:
#scrapper_kloop.df.append(pd.DataFrame(['lskdjfdsf'], columns=list(df.columns)), ignore_index=True)

In [20]:
df = scrapper_kloop.scrape('https://www.bbc.com/news/')

Loadedhttps://www.bbc.com/
Loadedhttps://www.bbc.com/news
Dobavil
Loadedhttps://www.bbc.com/sport


ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
#sys.getrecursionlimit()

In [22]:
scrapper_kloop.df

,title
0,BBC News Home
